## Part 3: Segmenting and Clustering Neighborhoods in Toronto
* Part 3 of Week 3 Assignment
* Task: Segmenting and Clustering Neighborhoods in Toronto
* Name: Hang Nguyen

In [5]:
# import needed packages
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 1. Scraping data from neighborhoods in Toronto
That is done in Part 1 of this assignment

In [6]:
# link to scrape information/data from neighborhoods in Toronto 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')

# initial list for table content
table_contents=[]

table = soup.find('table')

#extract data from table to list (table_contents)
for row in table.findAll('td'):
    # dictionary called cell having 3 keys PostalCode, Borough and Neighborhood
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        #As postal code contains upto 3 characters extract that using tablerow.p.text
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#Create a dataframe with list
df=pd.DataFrame(table_contents)

# clean the data
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business', 'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto', 'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### 2. Get the latitude and the longitude coordinates of each neighborhood
That is done in Part 2 of this assignment

In [7]:
#I download the Geospatial_Coordinates.csv file ( http://cocl.us/Geospatial_data ) to my local computer and call it from that
import sys
filepath = sys.path[0] + '/Geospatial_Coordinates.csv' 
df_postalcode = pd.read_csv(filepath)

df_postalcode = df_postalcode.rename(columns={'Postal Code':'PostalCode'})
#merge 2 dataframes
df_toronto = pd.merge(df,df_postalcode, on = 'PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### 3. Segmenting and Clustering Neighborhoods in Toronto

#### 3.1 Create a map of Toronto and have an oveview of Neighborhoods in Toronto

In [8]:
#Use geopy library to get the latitude and longitude values of Toronto, Ontario

import geocoder # import geocoder
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


In [9]:
#Create a map of Toronto with neighborhoods superimposed on top, using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.2 Create a map of Downtown Toroto
I decide to work with Downtown Toronto boground only. So I will segment and cluster only the neighborhoods in Downtown Toronto. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto.

In [10]:
# dataframe of Downtown Toronto data
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [14]:
# Let's get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [15]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Borough'], downtown_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [16]:
#Define Foursquare Credentials and Version
# To keep the privacy of my client_id and client_secret, I edited them after running the cell 
CLIENT_ID = '******************************************' # Foursquare ID
CLIENT_SECRET = '******************************************' # Foursquare Secret

VERSION = '20210406' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [17]:
# Let's see all neighborhoods in Downtown Toronto
downtown_data['Neighborhood']

0                             Regent Park, Harbourfront
1                              Garden District, Ryerson
2                                        St. James Town
3                                           Berczy Park
4                                    Central Bay Street
5                                              Christie
6                              Richmond, Adelaide, King
7     Harbourfront East, Union Station, Toronto Islands
8              Toronto Dominion Centre, Design Exchange
9                        Commerce Court, Victoria Hotel
10                       University of Toronto, Harbord
11            Kensington Market, Chinatown, Grange Park
12    CN Tower, King and Spadina, Railway Lands, Har...
13                                             Rosedale
14                          St. James Town, Cabbagetown
15               First Canadian Place, Underground city
16                                 Church and Wellesley
Name: Neighborhood, dtype: object

In [18]:
# I decide to explore the Neighborhood named 'Christie' just because the name is cute !
# The index of Christie is 5
#Get the neighborhood's latitude and longitude values.
neigh_latitude = downtown_data.loc[5, 'Latitude'] # neighborhood latitude value
neigh_longitude = downtown_data.loc[5, 'Longitude'] # neighborhood longitude value

neigh_name = downtown_data.loc[5, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of Christie are 43.669542, -79.4225637.


In [19]:
#Now, let's get the top 50 venues that are in Christie within a radius of 500 meters.
radius = 500
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neigh_latitude, neigh_longitude, VERSION, radius, LIMIT)

In [20]:
results = requests.get(url).json()

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
3,Scout and Cash Caffe,Café,43.667360,-79.419938
4,Starbucks,Coffee Shop,43.671530,-79.421400


In [23]:
# How many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


#### 3.3 Explore Neighborhoods in Downtown Toronto

In [24]:
# Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [26]:
#Let's check the size of the resulting dataframe
print(downtown_venues.shape)
downtown_venues.head()

(709, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [27]:
# Let's check how many venues were returned for each neighborhood
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",50,50,50,50,50,50
"Harbourfront East, Union Station, Toronto Islands",50,50,50,50,50,50


In [28]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 176 uniques categories.


#### 3.4 Analyze Each Neighborhood

In [29]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
#downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0

In [30]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.00,0.0000,0.00,0.0000,0.02,0.040000,0.000000,0.000000,0.00,0.02,0.02,0.040000,0.000000,0.02,0.02,0.0000,0.000000,0.0000,0.00,0.020000,0.00,0.00,0.00,0.00,0.000000,0.020000,0.0000,0.000000,0.04,0.000000,0.000000,0.02,0.04,0.080000,0.000000,0.000000,0.00,0.00,0.020000,0.00,0.02,0.000000,0.000000,0.02,0.00,0.000000,0.02,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.040000,0.00,0.00,0.02,0.000000,0.00,0.00,0.02,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.020000,0.02,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.000000,0.020000,0.02,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.0000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.040000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.0000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.0000,0.04,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.02,0.00,0.020000,0.02,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.125,0.125,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.0000,0.00,0.0000,0.00,0.000000,0.000000,0.062500,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.0625,0.000000,0.0625,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.062500,0.000000,0.

In [31]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+ hood +"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1              Bakery  0.04
2        Cocktail Bar  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.06
2                Café  0.06
3        Burger Joint  0.04
4  Italian Restaurant  0.04


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Athletics & Sports  0.06
4         Coffee Shop  0.06


----Church and Wellesley----
                 venue  freq
0  Japanese Restaurant  0.06
1          Coffee Shop  0.06
2     Sushi Restaurant  0.06
3       

In [32]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
# let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Restaurant,Beer Bar,Pharmacy,French Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Airport Food Court,Plane,Coffee Shop,Rental Car Location,Sculpture Garden
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Burger Joint,Italian Restaurant,New American Restaurant,Miscellaneous Shop,Modern European Restaurant,Bubble Tea Shop,Falafel Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Coffee Shop,Restaurant,Italian Restaurant,Nightclub,Candy Store,Baby Store
4,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Yoga Studio,Men's Store,Restaurant,Pizza Place,Burrito Place,Café,Caribbean Restaurant


#### 3.5 Cluster Neighborhoods

In [34]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 4, 3, 4, 0, 0, 4, 4, 4])

In [35]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge downtown_grouped with downtown_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Shoe Store,Historic Site
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Coffee Shop,Café,Cosmetics Shop,Ramen Restaurant,Theater,Bookstore,Clothing Store,Fast Food Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Coffee Shop,Cosmetics Shop,Creperie,Farmers Market,Seafood Restaurant,Gastropub,Park,Beer Bar,Department Store
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Restaurant,Beer Bar,Pharmacy,French Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Sandwich Place,Café,Burger Joint,Italian Restaurant,New American Restaurant,Miscellaneous Shop,Modern European Restaurant,Bubble Tea Shop,Falafel Restaurant


In [36]:
# let's visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.6 Examine Clusters

In [37]:
# Cluster 1
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0,downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Shoe Store,Historic Site
3,Berczy Park,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Seafood Restaurant,Farmers Market,Restaurant,Beer Bar,Pharmacy,French Restaurant
6,"Richmond, Adelaide, King",0,Coffee Shop,Café,Pizza Place,American Restaurant,Restaurant,Steakhouse,Concert Hall,Department Store,Burrito Place,Salon / Barbershop
8,"Toronto Dominion Centre, Design Exchange",0,Café,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Gastropub,Beer Bar,Japanese Restaurant,Museum,Concert Hall
9,"Commerce Court, Victoria Hotel",0,Hotel,Café,Coffee Shop,Restaurant,Seafood Restaurant,Gym,Gastropub,Japanese Restaurant,Deli / Bodega,American Restaurant
15,"First Canadian Place, Underground city",0,Café,Coffee Shop,Restaurant,Deli / Bodega,Seafood Restaurant,Hotel,American Restaurant,Gym,Concert Hall,Japanese Restaurant


In [38]:
# Cluster 2
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1,downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Rosedale,1,Park,Playground,Trail,Music Venue,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


In [39]:
# Cluster 3
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2,downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"CN Tower, King and Spadina, Railway Lands, Har...",2,Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Airport Food Court,Plane,Coffee Shop,Rental Car Location,Sculpture Garden


In [40]:
# Cluster 4
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3,downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,3,Grocery Store,Café,Park,Athletics & Sports,Coffee Shop,Restaurant,Italian Restaurant,Nightclub,Candy Store,Baby Store


In [41]:
# Cluster 5
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4,downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Garden District, Ryerson",4,Coffee Shop,Café,Cosmetics Shop,Ramen Restaurant,Theater,Bookstore,Clothing Store,Fast Food Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,St. James Town,4,Café,Coffee Shop,Cosmetics Shop,Creperie,Farmers Market,Seafood Restaurant,Gastropub,Park,Beer Bar,Department Store
4,Central Bay Street,4,Coffee Shop,Sandwich Place,Café,Burger Joint,Italian Restaurant,New American Restaurant,Miscellaneous Shop,Modern European Restaurant,Bubble Tea Shop,Falafel Restaurant
7,"Harbourfront East, Union Station, Toronto Islands",4,Aquarium,Coffee Shop,Hotel,Café,Park,Brewery,Plaza,Restaurant,Dessert Shop,Skating Rink
10,"University of Toronto, Harbord",4,Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Restaurant,Beer Bar,Beer Store,Sandwich Place
11,"Kensington Market, Chinatown, Grange Park",4,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Gaming Cafe,Brewery,Belgian Restaurant,Organic Grocery
14,"St. James Town, Cabbagetown",4,Coffee Shop,Pizza Place,Restaurant,Café,Italian Restaurant,Bakery,Park,Pub,Flower Shop,Market
16,Church and Wellesley,4,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Yoga Studio,Men's Store,Restaurant,Pizza Place,Burrito Place,Café,Caribbean Restaurant
